<a href="https://colab.research.google.com/github/vijaygwu/classideas/blob/main/BGBB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:

import numpy as np
import scipy
from scipy.optimize import differential_evolution
from scipy.special import loggamma
from scipy.optimize import minimize, Bounds
from scipy.special import gammaln

# Data
x = np.array([3, 5, 2])  # purchase history
x = [v/1000 for v in x] # Scale data
tx = np.array([4, 6, 3])  # timing of the last purchase
n = 6  # number of transaction opportunities
f = np.array([200, 300, 100])  # frequency of customers


# Define initial parameter guesses
params0 = [0.5, 0.5, 0.5, 0.5]

# Define the log-likelihood function
def ll(params, x, tx, n, f):
    # Unpack parameters
    r, alpha, a, b = params

    # Initialize log-likelihood
    log_likelihood = 0

    # Loop through each customer's data
    for xj, txj, fj in zip(x, tx, f):
        # Compute the probability of transaction
        p_transaction = r / (r + alpha)

        customer_ll = (
            xj * np.log(p_transaction) + (n - xj) * np.log(1 - p_transaction) +
            gammaln(a + b) - gammaln(a) - gammaln(b) + gammaln(xj + a) + gammaln(n - xj + b) - gammaln(n + a + b)
        )


        # Add to total log-likelihood
        log_likelihood += fj * customer_ll

    # Return the negative of the log-likelihood
    return -log_likelihood


# Specify the bounds using the Bounds class
bounds = Bounds([1e-4, 1e-4, 1e-4, 1e-4], [10, 100, 1, 1])

# Optimize using the trust-region reflective algorithm
opt_result = minimize(ll, params0, args=(x, tx, n, f), method='trust-constr', bounds=bounds)

# Perform the global optimization
result = differential_evolution(
    ll,
    bounds,
    args=(x, tx, n, f),
    strategy='best1bin',
    maxiter=1000,
    popsize=15,
    tol=0.01,
    mutation=(0.5, 1),
    recombination=0.7
)

# Display the estimated parameters
print(f"Global optimization estimated parameters: r = {result.x[0]}, alpha = {result.x[1]}, a = {result.x[2]}, b = {result.x[3]}")


# Based on the estimated parameters and the customer's history
def predict_alive(r, alpha, x, tx, n):
    p_alive = (r / (r + alpha))**x * ((r + alpha) / (r + alpha + n - tx))**(n - tx)
    return p_alive

# Predict the probability of the first customer being alive
p_alive_first_customer = predict_alive(result.x[0], result.x[1], result.x[2], result.x[3], n)
print(f"\nProbability of the first customer being alive: {p_alive_first_customer}")






Global optimization estimated parameters: r = 0.04961275395229879, alpha = 77.6063155810057, a = 0.03801772906115255, b = 1.0

Probability of the first customer being alive: 0.5534167681001817
